In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import *

In [2]:
QUOTE_DATE = '2023-06-30'
filepath_rawdata = f'../data/treasury_quotes_{QUOTE_DATE}.xlsx'
rawdata = pd.read_excel(filepath_rawdata,sheet_name='quotes')
rawdata.columns = rawdata.columns.str.upper()
rawdata.sort_values('TMATDT',inplace=True)
rawdata.set_index('KYTREASNO',inplace=True)

t_check = rawdata['CALDT'].values[0]
if rawdata['CALDT'].eq(t_check).all():
    t_current = t_check
else:
    warnings.warn('Quotes are from multiple dates.')
    t_current = None

display(f'Treasuries quoted on {QUOTE_DATE}.')

'Treasuries quoted on 2023-06-30.'

In [3]:
mats = get_maturity_delta(rawdata['TMATDT'],t_current=t_current)
targs = pd.Series(np.arange(.5,7.5,.5))

idbonds = list()
for tau in targs:
    idbonds.append((mats-tau).abs().idxmin())

metrics = rawdata.copy()[['TDATDT','TMATDT','TCOUPRT','TDYLD','TDDURATN']]
metrics.columns = ['issue date','maturity date','coupon rate','ytm','duration']
metrics['ytm'] *= 365
metrics['duration'] /= 365
metrics['maturity'] = get_maturity_delta(metrics['maturity date'], t_current)
metrics['price'] = (rawdata['TDBID'] + rawdata['TDASK'])/2 + rawdata['TDACCINT']
metrics = metrics[['issue date','maturity date','maturity','coupon rate','price','ytm']]

mets = metrics.loc[idbonds,:]
mets

,issue date,maturity date,maturity,coupon rate,price,ytm
KYTREASNO,,,,,,
207047,2016-12-31,2023-12-31,0.503765,2.250,98.515625,0.051858
207103,2017-06-30,2024-06-30,1.002053,2.000,96.722656,0.053243
207458,2019-12-31,2024-12-31,1.505818,1.750,95.054688,0.051301
207224,2018-06-30,2025-06-30,2.001369,2.750,95.882812,0.048702
207297,2018-12-31,2025-12-31,2.505133,2.625,95.171875,0.046273
207731,2021-06-30,2026-06-30,3.000684,0.875,90.023438,0.044119
207459,2019-12-31,2026-12-31,3.504449,1.750,91.539062,0.043281
207561,2020-06-30,2027-06-30,4.000000,0.500,86.148438,0.042581
207653,2020-12-31,2027-12-31,4.503765,0.625,85.335938,0.041879


In [4]:
outfile = '../data/selected_treasury_rates.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    mets.to_excel(writer, sheet_name= 'data', index=True)